import data

In [1]:
import pickle
with open('../pickles/data_cleaned.pickle', 'rb') as handle:
    data_cleaned = pickle.load(handle)

In [2]:
nosleep2020 = data_cleaned['RS_2020_nosleep']

In [3]:
confessions2020 = data_cleaned['RS_2020_confessions']

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten
from tqdm.notebook import tqdm
import swifter
import pandas as pd
import numpy as np

In [5]:
def LSTMlastLayer(row):
    import spacy
    from sentence_transformers import SentenceTransformer, util
    import torch
    import numpy as np
    spacy.prefer_gpu()
    
    print(row)
    
    nlp = spacy.load("en_core_web_lg")
    doc = nlp(row)
    
    embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

    sents = [sent.text for sent in doc.sents]
    # encode gives shape (# of sents, 768,)
    data = embedder.encode(sents)
    # reshape the data to shape (# of sents, 768, 1)
    data = data.reshape(data.shape[0],data.shape[1],-1)
    
    LSTMmodel = Sequential([
        LSTM(768, input_shape = data.shape[1:], return_sequences = True),
        Dropout(0.1),
        LSTM(768),
        Dropout(0.1, name="lastLayer")
    ])

    intermediate_layer_model = tf.keras.Model(inputs=LSTMmodel.input,
                                     outputs=LSTMmodel.get_layer('lastLayer').output)
    intermediate_output = intermediate_layer_model.predict(data)
    
    return np.average(intermediate_output,axis=0)

In [6]:
nosleep2020['LSTMvec'] = nosleep2020.selftext.swifter.progress_bar(enable=True).apply(LSTMlastLayer)

OutOfMemoryError: Out of memory allocating 821,797,376 bytes (allocated so far: 1,757,340,160 bytes).

In [17]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()